In [3]:
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity


#for

# Sample interaction data
data = [
    {"developer": "alice", "project": "projectA", "commits": 5, "pull_requests": 2, "stars": 1},
    {"developer": "alice", "project": "projectB", "commits": 2, "pull_requests": 0, "stars": 1},
    {"developer": "alice", "project": "projectC", "commits": 2, "pull_requests": 1, "stars": 3},
    {"developer": "bob", "project": "projectA", "commits": 3, "pull_requests": 1, "stars": 0},
    {"developer": "charlie", "project": "projectC", "commits": 1, "pull_requests": 0, "stars": 0},
    {"developer": "ask", "project": "projectA", "commits": 2, "pull_requests": 1, "stars": 3}
]

# Convert to DataFrame
df = pd.DataFrame(data)

# Define weights
w_commits, w_prs, w_stars = 2.0, 0.5, 4.0

# Compute interaction scores
df["interaction"] = (
    w_commits * df["commits"] +
    w_prs * df["pull_requests"] +
    w_stars * df["stars"]
)

# Create user-item matrix
user_item_matrix = df.pivot_table(index='developer', columns='project', values='interaction', fill_value=0)

# Convert to sparse matrix
sparse_matrix = csr_matrix(user_item_matrix.values)

# Compute cosine similarity between developers
similarities = cosine_similarity(sparse_matrix)

# Convert to DataFrame for readability
similarity_df = pd.DataFrame(similarities, index=user_item_matrix.index, columns=user_item_matrix.index)

# Function to recommend projects
def recommend_projects(target_developer, user_item_matrix, similarity_df, top_n=2):
    if target_developer not in user_item_matrix.index:
        return f"Developer {target_developer} not found."
    
    # Get similar developers
    similar_devs = similarity_df[target_developer].drop(target_developer).sort_values(ascending=False)

    # Get projects the target developer has already interacted with
    seen_projects = set(user_item_matrix.loc[target_developer][user_item_matrix.loc[target_developer] > 0].index)

    # Get projects from similar developers
    recommendations = {}
    for similar_dev, similarity_score in similar_devs.items():
        similar_projects = user_item_matrix.loc[similar_dev][user_item_matrix.loc[similar_dev] > 0].index
        for project in similar_projects:
            if project not in seen_projects:
                recommendations[project] = recommendations.get(project, 0) + similarity_score

    # Sort recommendations by strength and return top N
    sorted_recommendations = sorted(recommendations.items(), key=lambda x: x[1], reverse=True)
    return [project for project, score in sorted_recommendations[:top_n]]

# Example: Recommend projects for Alice
print("Recommended projects for bob:", recommend_projects("bob", user_item_matrix, similarity_df))


Recommended projects for bob: ['projectB', 'projectC']
